In [16]:
%cd ..

/home/lonya/doc/deeph/YIG/configuration


In [2]:
import os
import subprocess as sp
import time

import numpy as np
from pymatgen.core.structure import Structure

cwd_dir = '/home/lonya/doc/deeph/YIG'

In [15]:
!ls

In [5]:
np.random.seed(42)

os.chdir(cwd_dir)

structure = Structure.from_file('YIG.cif')
structure_supercell = Structure(structure.lattice.matrix,
                                structure.atomic_numbers,
                                structure.frac_coords,
                                coords_are_cartesian=False,
                                to_unit_cell=True)
structure_supercell.make_supercell([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
N_supercell = len(structure_supercell)

pseudo_dir = os.path.join(cwd_dir, 'pseudo')
orbital_dir = os.path.join(cwd_dir, 'orbital')


root_dir = 'configuration'
for index_perturb in range(100):
    work_dir = os.path.join(cwd_dir, root_dir, str(index_perturb))
    os.makedirs(work_dir, exist_ok=True)
    os.chdir(work_dir)

    origin_cart_coords = structure_supercell.cart_coords

    disorder_x = (np.random.rand(N_supercell, 1) - 0.5) * 0.2 # -0.1 ~ 0.1
    disorder_y = (np.random.rand(N_supercell, 1) - 0.5) * 0.2
    disorder_z = (np.random.rand(N_supercell, 1) - 0.5) * 0.2

    cart_coords = origin_cart_coords + np.concatenate([disorder_x, disorder_y, disorder_z], axis=-1)

    structure_supercell_perturb = Structure(structure_supercell.lattice.matrix,
                                            structure_supercell.atomic_numbers,
                                            cart_coords,
                                            coords_are_cartesian=True,
                                            to_unit_cell=True)
    
    numbers = structure_supercell_perturb.atomic_numbers
    NUM_ATOM = len(numbers)
    lattice = structure_supercell_perturb.lattice.matrix
    frac_coords = structure_supercell_perturb.frac_coords
    origin_frac_coords = structure_supercell.frac_coords
    Y_frac_coords_str = ''
    FeO_frac_coords_str = ''
    FeT_frac_coords_str = ''
    O_frac_coords_str = ''
    for i in range(NUM_ATOM):
            if numbers[i] == 39:
                Y_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2]:.16f} 0 0 0\n'
            elif numbers[i] == 26:
                sum = origin_frac_coords[i, 0] + origin_frac_coords[i, 1] + origin_frac_coords[i, 2]
                while sum > 0.24:
                    sum -= 0.25
                if sum > 0.12:
                    FeT_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2]:.16f} 0 0 0\n'
                else:
                    FeO_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2]:.16f} 0 0 0\n'
            elif numbers[i] == 8:
                O_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2]:.16f} 0 0 0\n'
            else:
                print(numbers[i])
                raise ValueError("numbers[i] error")

    structure_supercell_perturb.to('poscar', 'POSCAR')

    in_file = fr"""ATOMIC_SPECIES
Y 88.906 Y_ONCV_PBE-1.0.upf
FeO 55.845 Fe_ONCV_PBE-1.0.upf
FeT 55.845 Fe_ONCV_PBE-1.0.upf
O 15.999 O_ONCV_PBE-1.0.upf

NUMERICAL_ORBITAL
Y_gga_8au_100Ry_4s2p2d1f.orb
Fe_gga_7au_100Ry_4s2p2d1f.orb
Fe_gga_7au_100Ry_4s2p2d1f.orb
O_gga_6au_100Ry_2s2p1d.orb

LATTICE_CONSTANT
1.8897259886 # 1.8897259886 Bohr = 1.0 Angstrom

LATTICE_VECTORS
{lattice[0, 0]:.16f} {lattice[0, 1]:.16f} {lattice[0, 2]:.16f}
{lattice[1, 0]:.16f} {lattice[1, 1]:.16f} {lattice[1, 2]:.16f}
{lattice[2, 0]:.16f} {lattice[2, 1]:.16f} {lattice[2, 2]:.16f}

ATOMIC_POSITIONS
Direct

Y
0.0
24
{Y_frac_coords_str}

FeO
-5.0
16
{FeO_frac_coords_str}

FeT
5.0
24
{FeT_frac_coords_str}

O
0.0
96
{O_frac_coords_str}
"""
    with open('STRU', 'w') as f:
        f.write(in_file)

    in_file = fr"""K_POINTS
0
Gamma
1 1 1 0 0 0
"""
    with open('KPT', 'w') as f:
        f.write(in_file)

    in_file = fr"""INPUT_PARAMETERS
pseudo_dir {pseudo_dir}
orbital_dir {orbital_dir}
ntype                   4
nspin                   2
calculation             scf
basis_type              lcao
ecutwfc                 100
scf_thr                 1.0e-4
scf_nmax                100
gamma_only              0

ks_solver    genelpa
smearing_method         gaussian
smearing_sigma          0.05

mixing_type             pulay
mixing_beta             0.4

dft_plus_u    0
orbital_corr    -1 2 2 -1
hubbard_u    0.0 5.0 5.0 0.0

out_mat_hs2       1
"""

    with open('INPUT', 'w') as f:
        f.write(in_file)

In [6]:
os.environ['OMP_NUM_THREADS'] = '8'

for index_abacus in range(100):
    work_dir = os.path.join(cwd_dir, 'configuration', str(index_abacus))
    os.chdir(work_dir)
    sp.run(['mpirun', '-np', '1', 'abacus'])
    #sp.run(['mpirun', '-np', '2', 'abacus'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)





                                                                                     
                              ABACUS v3.6.0

               Atomic-orbital Based Ab-initio Computation at UStc                    

                     Website: http://abacus.ustc.edu.cn/                             
               Documentation: https://abacus.deepmodeling.com/                       
                  Repository: https://github.com/abacusmodeling/abacus-develop       
                              https://github.com/deepmodeling/abacus-develop         
                      Commit: 059fc16 (Sat Apr 13 12:08:33 2024 +0800)

 Tue May 14 17:12:32 2024
 MAKE THE DIR         : OUT.ABACUS/
 RUNNING WITH DEVICE  : CPU / 13th Gen Intel(R) Core(TM) i9-13900K

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 Pseudopotentials with additional electrons can yield (more) accurate outcomes, but may be less efficient.
 If you're confident th

KeyboardInterrupt: 